In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

In [2]:
spark=SparkSession.builder.appName("Decision_Tree").getOrCreate()

24/11/06 01:10:35 WARN Utils: Your hostname, Anants-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.141.67 instead (on interface en0)
24/11/06 01:10:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/06 01:10:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/06 01:10:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/06 01:10:36 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


1) Loading Dataset

In [4]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col
data_without_header=spark.read.options(header=False ,inferSchema=True).csv("covtype.csv")
data_without_header.printSchema()

colnames =  ["Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways", "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm", "Horizontal_Distance_To_Fire_Points"]
colnames = colnames + [f'Wilderness_Area_{i}' for i in range(4)]
colnames = colnames + [f'Soil_Type{i}' for i in range(40)] + ['Cover_Type']
data2 = data_without_header.toDF(*colnames)
data2= data2.withColumn('Cover_Type', col('Cover_Type').cast(DoubleType()))
data2.show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

24/11/06 01:10:40 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+-----------------+-----------------+-----------------+-----------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm|Horizontal_Distance_To_Fire_Points|Wilderness_Area_0|Wilderness_Area_1|Wilderness_Ar

In [5]:
data=spark.read.options(header=True ,inferSchema=True).csv("covtype.csv")
data.printSchema()
data.head()

root
 |-- Elevation: integer (nullable = true)
 |-- Aspect: integer (nullable = true)
 |-- Slope: integer (nullable = true)
 |-- Horizontal_Distance_To_Hydrology: integer (nullable = true)
 |-- Vertical_Distance_To_Hydrology: integer (nullable = true)
 |-- Horizontal_Distance_To_Roadways: integer (nullable = true)
 |-- Hillshade_9am: integer (nullable = true)
 |-- Hillshade_Noon: integer (nullable = true)
 |-- Hillshade_3pm: integer (nullable = true)
 |-- Horizontal_Distance_To_Fire_Points: integer (nullable = true)
 |-- Wilderness_Area1: integer (nullable = true)
 |-- Wilderness_Area2: integer (nullable = true)
 |-- Wilderness_Area3: integer (nullable = true)
 |-- Wilderness_Area4: integer (nullable = true)
 |-- Soil_Type1: integer (nullable = true)
 |-- Soil_Type2: integer (nullable = true)
 |-- Soil_Type3: integer (nullable = true)
 |-- Soil_Type4: integer (nullable = true)
 |-- Soil_Type5: integer (nullable = true)
 |-- Soil_Type6: integer (nullable = true)
 |-- Soil_Type7: integer

Row(Elevation=2596, Aspect=51, Slope=3, Horizontal_Distance_To_Hydrology=258, Vertical_Distance_To_Hydrology=0, Horizontal_Distance_To_Roadways=510, Hillshade_9am=221, Hillshade_Noon=232, Hillshade_3pm=148, Horizontal_Distance_To_Fire_Points=6279, Wilderness_Area1=1, Wilderness_Area2=0, Wilderness_Area3=0, Wilderness_Area4=0, Soil_Type1=0, Soil_Type2=0, Soil_Type3=0, Soil_Type4=0, Soil_Type5=0, Soil_Type6=0, Soil_Type7=0, Soil_Type8=0, Soil_Type9=0, Soil_Type10=0, Soil_Type11=0, Soil_Type12=0, Soil_Type13=0, Soil_Type14=0, Soil_Type15=0, Soil_Type16=0, Soil_Type17=0, Soil_Type18=0, Soil_Type19=0, Soil_Type20=0, Soil_Type21=0, Soil_Type22=0, Soil_Type23=0, Soil_Type24=0, Soil_Type25=0, Soil_Type26=0, Soil_Type27=0, Soil_Type28=0, Soil_Type29=1, Soil_Type30=0, Soil_Type31=0, Soil_Type32=0, Soil_Type33=0, Soil_Type34=0, Soil_Type35=0, Soil_Type36=0, Soil_Type37=0, Soil_Type38=0, Soil_Type39=0, Soil_Type40=0, Cover_Type=5)

In [6]:
train_data,test_data=data.randomSplit([0.8,0.2],seed=42)
train_data.cache()
test_data.cache()
train_data.show()

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+----------------+----------------+----------------+----------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm|Horizontal_Distance_To_Fire_Points|Wilderness_Area1|Wilderness_Area2|Wilderness_Area3|Wi

2) preprocessing data

In [8]:
from pyspark.ml.feature import StringIndexer, VectorAssembler

print(f"rows count before removing missing rows:{data.count()}")
# Handling missing values
# Dropping rows with missing values
data = data.dropna()
print(f"rows count after removing missing rows:{data.count()}")
# # Handling categorical features using StringIndexer
# categorical_columns = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'income']
# indexers = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in categorical_columns]

# # Create a pipeline for preprocessing
# pipeline = Pipeline(stages=indexers + [assembler])
# df_prepared = pipeline.fit(df_clean).transform(df_clean)

rows count before removing missing rows:581012


rows count after removing missing rows:581012


In [9]:
# # Assemble features into a feature vector
# assembler = VectorAssembler(
#     inputCols=['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week'] + [col+"_index" for col in categorical_columns[:-1]],
#     outputCol='features'
# )
cols=data.columns
vector_assembler=VectorAssembler(inputCols=cols[:-1],outputCol='features')
assembled_train_data=vector_assembler.transform(train_data)
assembled_train_data.select(["features","Cover_Type"]).show(5)

+--------------------+----------+
|            features|Cover_Type|
+--------------------+----------+
|(54,[0,1,2,3,4,5,...|         6|
|(54,[0,1,2,5,6,7,...|         6|
|(54,[0,1,2,3,4,5,...|         6|
|(54,[0,1,2,3,4,5,...|         6|
|(54,[0,1,2,3,4,5,...|         6|
+--------------------+----------+
only showing top 5 rows



In [10]:
from pyspark.ml.classification import DecisionTreeClassifier
classifier=DecisionTreeClassifier(seed=1234, featuresCol='features',labelCol="Cover_Type",predictionCol='predictions')
model=classifier.fit(assembled_train_data)
print(model.toDebugString)

24/11/06 01:10:47 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


DecisionTreeClassificationModel: uid=DecisionTreeClassifier_7fd7b76014fb, depth=5, numNodes=49, numClasses=8, numFeatures=54
  If (feature 0 <= 3036.5)
   If (feature 0 <= 2545.5)
    If (feature 10 <= 0.5)
     If (feature 0 <= 2413.5)
      If (feature 3 <= 15.0)
       Predict: 4.0
      Else (feature 3 > 15.0)
       Predict: 3.0
     Else (feature 0 > 2413.5)
      Predict: 3.0
    Else (feature 10 > 0.5)
     If (feature 9 <= 5460.5)
      Predict: 2.0
     Else (feature 9 > 5460.5)
      If (feature 5 <= 524.5)
       Predict: 2.0
      Else (feature 5 > 524.5)
       Predict: 5.0
   Else (feature 0 > 2545.5)
    If (feature 15 <= 0.5)
     If (feature 17 <= 0.5)
      Predict: 2.0
     Else (feature 17 > 0.5)
      If (feature 0 <= 2678.5)
       Predict: 3.0
      Else (feature 0 > 2678.5)
       Predict: 2.0
    Else (feature 15 > 0.5)
     If (feature 9 <= 1439.0)
      If (feature 7 <= 216.5)
       Predict: 2.0
      Else (feature 7 > 216.5)
       Predict: 3.0
     Else (

In [11]:
predictions=model.transform(assembled_train_data)
predictions.select(['Cover_Type','predictions','probability']).show(5,truncate=False)

+----------+-----------+-------------------------------------------------------------------------------------------------+
|Cover_Type|predictions|probability                                                                                      |
+----------+-----------+-------------------------------------------------------------------------------------------------+
|6         |3.0        |[0.0,0.0,0.02545064185713647,0.6457038064140985,0.05770004920159234,0.0,0.2711455025271727,0.0]  |
|6         |4.0        |[0.0,0.0,0.024322830292979547,0.28745163073521285,0.4847982310668878,0.0,0.20342730790491984,0.0]|
|6         |3.0        |[0.0,0.0,0.02545064185713647,0.6457038064140985,0.05770004920159234,0.0,0.2711455025271727,0.0]  |
|6         |3.0        |[0.0,0.0,0.02545064185713647,0.6457038064140985,0.05770004920159234,0.0,0.2711455025271727,0.0]  |
|6         |3.0        |[0.0,0.0,0.02545064185713647,0.6457038064140985,0.05770004920159234,0.0,0.2711455025271727,0.0]  |
+----------+----

4) Evaluation Metrics

In [13]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator=MulticlassClassificationEvaluator(labelCol='Cover_Type',predictionCol='predictions')

f1score=evaluator.setMetricName('f1').evaluate(predictions)
accuracy_score=evaluator.setMetricName('accuracy').evaluate(predictions)
precision_score=evaluator.setMetricName('weightedPrecision').evaluate(predictions)
recall_score=evaluator.setMetricName('weightedRecall').evaluate(predictions)

print(f"precision score:{precision_score}")
print(f"f1 score:{f1score}")
print(f"accuracy score:{accuracy_score}")
print(f"recall score:{recall_score}")

precision score:0.6796868512937382
f1 score:0.6810376100570322
accuracy score:0.6994398765705727
recall score:0.6994398765705728


columns are predicated classes, and rows are actual classes

In [15]:
confusion_matrix=predictions.groupBy('Cover_Type').pivot('predictions',range(1,9))\
                    .count()\
                    .na.fill(0.0)\
                    .orderBy('Cover_Type')

confusion_matrix.show()

+----------+------+------+-----+---+---+---+----+---+
|Cover_Type|     1|     2|    3|  4|  5|  6|   7|  8|
+----------+------+------+-----+---+---+---+----+---+
|         1|109014| 55859|   89|  0|  0|  0|4608|  0|
|         2| 40587|180896| 4374| 44| 27|  0| 640|  0|
|         3|     0|  2983|25091|520|  0|  0|   0|  0|
|         4|     0|     0| 1327|877|  0|  0|   0|  0|
|         5|     0|  7216|  235|  0| 68|  0|   0|  0|
|         6|     0|  3040|10492|368|  0|  0|   0|  0|
|         7|  7201|    66|    0|  0|  0|  0|9097|  0|
+----------+------+------+-----+---+---+---+----+---+



HyperTuning

In [17]:
from pyspark.ml.tuning import TrainValidationSplit,ParamGridBuilder

classifier=DecisionTreeClassifier(seed=1234, featuresCol='features',labelCol="Cover_Type",predictionCol='predictions')
evaluator=MulticlassClassificationEvaluator(labelCol='Cover_Type',predictionCol='predictions',metricName='accuracy')
parameter_grid=(ParamGridBuilder()\
               .addGrid(model.impurity,['gini','entropy'])\
               .addGrid(model.maxDepth,[1,20])\
               .addGrid(model.maxBins,[40,300])\
               .addGrid(model.minInfoGain,[0.0,0.05])\
               .build())

validator=TrainValidationSplit(estimator=classifier,
                                estimatorParamMaps=parameter_grid,
                              evaluator=evaluator,seed=1234,trainRatio=0.9)
validator_model=validator.fit(assembled_train_data)

In [18]:
best_model=validator_model.bestModel
best_model

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_1a54c6bafb41, depth=5, numNodes=49, numClasses=8, numFeatures=54

In [19]:
metrics=validator_model.validationMetrics
params=validator_model.getEstimatorParamMaps()
metrics.sort(reverse=True)
print(metrics[0])
print(params[0])

0.6980126759050381
{Param(parent='DecisionTreeClassifier_7fd7b76014fb', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini', Param(parent='DecisionTreeClassifier_7fd7b76014fb', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 1, Param(parent='DecisionTreeClassifier_7fd7b76014fb', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 40, Param(parent='DecisionTreeClassifier_7fd7b76014fb', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0}


In [41]:
spark.stop()